 # Segmenting and Clustering Neighborhoods in Toronto

In [85]:
#import packages
import pandas as pd
import requests
from bs4 import BeautifulSoup

Now we scrap Toronto postalcodes from Wikipedia

In [86]:
#wikipedia link
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [87]:
#set up beautiful soup
bs=BeautifulSoup(url,'lxml')

since the zipcodes are displayed in a table, we pass "table" statement in the BeautifulSoup

In [88]:
#scrap postalcode table
postable=bs.find('table',{'class':'wikitable sortable'})

In [89]:
#convert the ziptable into a pandas dataframe
posdf=pd.read_html(str(postable),header=0)
posdf=pd.DataFrame(posdf[0])

In [90]:
#filter out the borough that are "not assigned"
posdf.drop(posdf[posdf.Borough=='Not assigned'].index, inplace=True)

In [91]:
# deal with repeated neighborhoods
posdf=posdf.groupby("Postcode").agg(lambda x:','.join(set(x)))
posdf.reset_index(inplace = True)

In [92]:
#rename column
posdf.rename(columns={"Postcode":"Postal Code"},inplace=True)

In [93]:
posdf.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill"
2,M1E,Scarborough,"Guildwood,West Hill,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [94]:
print('There are {} rows in the dataframe'.format(posdf.shape[0]))

There are 103 rows in the dataframe


# Use the Geocoder package to get the latitude and longitude

In [95]:
#download
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [96]:
#read teh spatial data into csv with pandas
geo=pd.read_csv('Geospatial_Coordinates.csv')
geo.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [97]:
print('There are {} rows in the geo dataframe'.format(geo.shape[0]))

There are 103 rows in the geo dataframe


geo dataframe has the same number of rows with posdf dataframe

In [99]:
#check if the number of rows from two dataframes are matched
pre_exam=posdf['Postal Code'].isin(geo['Postal Code'])
pre_exam.unique()

array([ True], dtype=bool)

Yes they matched.

In [100]:
#sort postal code before merging
posdf.sort_values(by='Postal Code',ascending=True,axis=0,inplace=True)
geo.sort_values(by='Postal Code',ascending=True,axis=0,inplace=True)

merged=pd.merge(posdf, geo, on=posdf['Postal Code'],right_index=True, left_index=True)

In [101]:
#check column names
merged.columns

Index(['Postal Code_x', 'Borough', 'Neighbourhood', 'Postal Code_y',
       'Latitude', 'Longitude'],
      dtype='object')

In [102]:
#drop duplicated columns and rename Postal Code column
merged.drop('Postal Code_y',axis=1,inplace=True)
merged.rename(columns={'Postal Code_x':'Postal Code'},inplace=True)

In [103]:
merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,West Hill,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [104]:
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize 
!conda install -c conda-forge folium=0.5.0 --yes 
import folium
print('installation finished')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
installation finished


In [105]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [106]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged['Latitude'], merged['Longitude'], merged['Borough'], merged['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto